# 7.Result [Task1 , Task2].ipynb

In [ ]:
import os
import pandas as pd
import geopandas as gpd
import json
import pathlib
import folium
from folium.plugins import MousePosition
from shapely.geometry import Point, Polygon

In [ ]:
input_path = pathlib.Path('./data')

## [과제1] 어린이 보호구역 외 어린이 교통사고 위험지역 20개소 선정 결과

In [ ]:
input_path = pathlib.Path('./data')

In [ ]:
어린이교통사고_격자 = gpd.read_file(input_path.joinpath('2.오산시_어린이교통사고_격자.geojson'),encoding="utf-8")
어린이보호구역 = pd.read_csv(input_path.joinpath('9.오산시_어린이보호구역.csv'),encoding="utf-8")
선정결과 = pd.read_csv(input_path.joinpath('ada_top_20.csv'))
선정결과.head(2)

> __선정결과 데이터에 geometry object 정보 추가__

In [ ]:
index=[]

for i,row in enumerate(선정결과.iloc):
    for j,row2 in enumerate(어린이교통사고_격자.iloc):
        if row.gid == row2.gid:
            index.append(j)            
선정결과 = 어린이교통사고_격자.iloc[index]

> __기존 어린이 보호구역과 함께 비교하여 상세 좌표 선정__

In [ ]:
# 위/경도 column 추가
선정결과[['centroid']] = 선정결과.geometry.centroid
선정결과['lon'] = 선정결과.centroid.x
선정결과['lat'] = 선정결과.centroid.y

In [ ]:
center=[37.164584, 127.050451]
zoom =12.5
protect_spot = 어린이보호구역[['시설종류','보호구역_경도','보호구역_위도']]

m = folium.Map(location=center, tiles='openstreetmap', zoom_start=zoom) 
folium.GeoJson(선정결과['geometry'].iloc[8]).add_to(m) # iloc값 바꿔가면서 명명했음 (구글 지도도 함께 참고)
for i in protect_spot.index:
    folium.Marker(
        location = 어린이보호구역.loc[i, ['보호구역_위도', '보호구역_경도']],
        radius = 300,
        tooltip = 어린이보호구역.loc[i,'시설명']
    ).add_to(m)

for i in protect_spot.index:
    folium.Circle(
        location = 어린이보호구역.loc[i, ['보호구역_위도', '보호구역_경도']],
        radius = 300,
        width = 0.5,
        color='green',
        tooltip = 어린이보호구역.loc[i,'시설종류']
    ).add_to(m)
MousePosition().add_to(m)
m

# 위험 구역의 중심점은 사고격자의 중심점
# 주로 사고는 도로에서 일어남, 하지만 위험 격자가 도로에 살짝 걸친경우, 예외적으로 반경의 중심점을 
# 사고격자와 살짝 걸치는도로의 중심으로 놓았음 (9순위 위험구역) 

# 위험구역격자와 어린이 보호구역이 겹치면 반경 50 / 안겹치면 반경 100

# iloc[0] : 원동주공 아파트 앞 경기대로 , 50
# iloc[1] : 교육지원청 사거리 , 50
# iloc[2] : 오산시청앞 삼거리, 100
# iloc[3] : 남촌오거리,  100
# iloc[4] : 오산비뇨기과 의원 부근 성호대로, 100
# iloc[5] : 경기동로 삼거리???  100
# iloc[6] : 말머리로 / 동부대로 교차사거리 (TH 모터스 부근) 100
# iloc[7] : 고현 아이파크 부근 남부대로 사거리  100
# iloc[8] : 예다원 모텔 부근 경기대로  50     >>  도로가 위험격자 끝에 걸쳤기에, 도로 중심으로 좌표 재조정 (127.05739,37.17818)
# iloc[9]  : 세교주유소 부근 경기대로  100  
# iloc[10] : 원동 프루지오 아파트 부근 사거리  50 
# iloc[11] : 롯데리아 오산세교 DT점 부근   100
# iloc[12] : 오산역 환승센터 부근  100
# iloc[13] : 오산대학교 앞 궐동로  50
# iloc[14] : 오산시청 부근 성호대로  100
# iloc[15] : 현대블루핸즈 가수점 부근 삼거리 50
# iloc[16] : 세마동 주민센터 부근  100
# iloc[17] : 지석묘진 일대  100
# iloc[18] : 한일아파트 부근 경기대로  100
# iloc[19] : 잔다리마을 2단지 일대 수목원로  50

# [과제2] 기존 어린이 보호구역 중 교통안전시설물 우선 설치 지역 20개소 선정 결과

In [ ]:
CCTV설치현황 = pd.read_csv(input_path.joinpath('20.오산시_CCTV설치현황.csv'),encoding="utf-8")
횡단보도 = gpd.read_file(input_path.joinpath('17.오산시_횡단보도.geojson'),encoding="utf-8")
신호등 = gpd.read_file(input_path.joinpath('19.오산시_신호등.geojson'),encoding="utf-8")
상세도로망 = gpd.read_file(input_path.joinpath('23.오산시_상세도로망_LV6.geojson'),encoding="utf-8")
과속방지턱 =  pd.read_csv(input_path.joinpath('18.오산시_과속방지턱표준데이터.csv'))
법정경계_읍면동 = gpd.read_file(input_path.joinpath('33.오산시_법정경계(읍면동).geojson'),encoding="utf-8")
상세도로망 = gpd.read_file(input_path.joinpath('23.오산시_상세도로망_LV6.geojson'),encoding="utf-8")
선정구역 = pd.read_csv(input_path.joinpath('data_for_task2_20kidszones.csv'),encoding="utf-8")
도로명주소_건물 = gpd.read_file(input_path.joinpath('27.오산시_도로명주소_건물.geojson'))

> __시각화를 위한 별도 전처리__

In [ ]:
상세도로망 = 상세도로망[상세도로망.road_name.str.contains('고속도로') == False]
상세도로망= 상세도로망.reset_index()
상세도로망_최대속도30 = 상세도로망[상세도로망.max_speed == '30']
상세도로망['width']=상세도로망.width.astype(int) * 3.5
도로명주소_건물=도로명주소_건물[(도로명주소_건물['BDTYP_CD'] =='01001')|(도로명주소_건물['BDTYP_CD'] =='02001')|(도로명주소_건물['BDTYP_CD'] =='01003')]
도로명주소_건물 = 도로명주소_건물.reset_index()
acc2 = 어린이교통사고_격자[어린이교통사고_격자['accident_cnt'] >= 2 ]

In [ ]:
with open(input_path.joinpath('33.오산시_법정경계(읍면동).geojson'),encoding="utf=8") as f:
    geo_data = json.loads(f.read())
    f.close()
with open(input_path.joinpath('23.오산시_상세도로망_LV6.geojson'),encoding="utf=8") as f:
    geo_data2 = json.loads(f.read())
    f.close()
    
center=[37.164584, 127.050451]
zoom =12.5

m = folium.Map(location=center, zoom_start=zoom)

folium.GeoJson(
    geo_data,
    name = "동 경계",
    style_function=lambda feature: {
        'fillColor': '#ADFF2F',
        'color': '#7CFC00',
        'weight': 2,
        'dashArray': '5, 5'
    }
).add_to(m)

folium.GeoJson(
    도로명주소_건물[도로명주소_건물.BDTYP_CD == '01001'],
    name = '단독주택',
    style_function=lambda feature: {
        'color': '#FF6666',
        'weight': 3,
        'opacity':0.7,
        'dashArray': '1,2,1'
    },
    tooltip=folium.GeoJsonTooltip(fields=['BDTYP_CD'], aliases = ['단독주택'])
).add_to(m)

folium.GeoJson(
    도로명주소_건물[도로명주소_건물.BDTYP_CD == '02001'],
    name = '아파트',
    style_function=lambda feature: {
        'color': '#0099FF',
        'weight': 3,
        'opacity':0.7,
        'dashArray': '1,2,1'
    },
    tooltip=folium.GeoJsonTooltip(fields=['BDTYP_CD'], aliases = ['아파트'])
).add_to(m)

folium.GeoJson(
    도로명주소_건물[도로명주소_건물.BDTYP_CD == '01003'],
    name = '다가구주택',
    style_function=lambda feature: {
        'color': '#FF6600',
        'weight': 3,
        'opacity':0.7,
        'dashArray': '1,2,1'
    },
    tooltip=folium.GeoJsonTooltip(fields=['BDTYP_CD'], aliases = ['다가구주택'])
).add_to(m)

for i in 어린이보호구역.index:
    folium.Circle(
        location = 어린이보호구역.loc[i, ['보호구역_위도', '보호구역_경도']],
        radius = 300,
        stroke=False,
        fill=True,
        fill_color='blue',
        fill_opacity=0.4,
        tooltip = 어린이보호구역.loc[i,'시설명']+',' + '도로폭:'+str(어린이보호구역.loc[i,'보호구역도로폭'])
    ).add_to(m)
    
for i in 과속방지턱.index:
    folium.Marker(
        location = 과속방지턱.loc[i, ['설치위치_위도', '설치위치_경도']],
        icon=folium.Icon(color = 'lightred',icon = 'star', prefix='fa'),
        tooltip = 과속방지턱.loc[i,'설치장소명']
    ).add_to(m)
    
    for i in 선정구역.index:
        folium.Marker(
        location = 선정구역.loc[i, ['보호구역_위도', '보호구역_경도']],
        tooltip = 선정구역.loc[i,'시설명'],
        icon=folium.Icon(color = 'darkgreen',icon = 'flag'),
    ).add_to(m)

folium.GeoJson(acc2['geometry'],
              style_function = lambda x: {
                  'color': 'yellow',
                  'weight': 1.5,
                  'fillOpacity': 0.3,
              }).add_to(m)

folium.GeoJson(
    geo_data2,
    name = "상세도로",
    style_function=lambda feature: {
        'fillColor': '#778899',
        'color': '#808080',
        'weight': 2,
        'dashArray': '5, 5'
    },
    tooltip=folium.GeoJsonTooltip(fields=['max_speed','width'], aliases = ['최대속도','도로폭'])   
).add_to(m)
folium.GeoJson(
    상세도로망_최대속도30,
    name = '상세도로망_최대속도30',
    style_function=lambda feature: {
        'color': '#FF69B4',
        'weight': 3,
        'opacity':0.7,
        'dashArray': '1,2,1'
    }
).add_to(m)

MousePosition().add_to(m)
m


# 어린이보호구역,최대속도30,도로폭,과속방지턱,어린이보호구역 선정20곳,교통사고격자 지도 데이터를 이용하여 
# folium플러그인인 마우스오버를 활용
# 선정20곳 지도 데이터를 보며 각 저장물들이 필요한 지역을 좌표로 찍고 구글지도를 이용해 로드맵으로 확인후 최종위치 선정

# 최대속도 60~80으 도로가 속도가 급격하게 줄어드는 지역 혹은 사고가 잦은 경우 미끄럼방지턱을 설치
# 도로폭이 평균이하인 8이면 과속방지턱, 이상이면 과속단속CCTV, 사고격자근처거나 차량이 인도와가까워 위험한곳은 가드휀스

# iloc[0] : 금암초등학교 , 과속단속CCTV
# iloc[1] : 세교복지타운어린이집 , 가드휀스
# iloc[2] : 시립가온어린이집, 과속단속CCTV
# iloc[3] : 동화마을유치원,  과속방지턱
# iloc[4] : 키즈청호어린이집, 교통안전표지판(어린이보호구역)
# iloc[5] : 삼미초등학교, 가드휀스
# iloc[6] : 시립세마어린이집, 가드휀스
# iloc[7] : 광성초등학교, 과속단속CCTV
# iloc[8] : 운산초등학교, 과속방지턱
# iloc[9]  : 지예뜰유치원, 과속방지턱
# iloc[10] : 이림유치원, 교통안전표지판(어린이보호구역) 
# iloc[11] : 수청초등학교, 미끄럼방지
# iloc[12] : 로뎀장애아전담 어린이집, 과속방지턱,교통안전표지판
# iloc[13] : 지움 어린이집, 교통안전표지판(어린이보호구역)
# iloc[14] : 지구촌어린이집, 과속방지턱
# iloc[15] : 영락어린이집, 가드휀스
# iloc[16] : 구세군샛별어린이집, 과속방지턱
# iloc[17] : 해에게서소년에게어린이집, 불법주정차CCTV
# iloc[18] : 자람터어린이집, 과속단속CCTV
# iloc[19] : 시립생명숲어린이집, 과속방지턱